In [1]:
import pandas as pd
import re
import numpy as np

In [2]:
df = pd.read_csv('./review_total_all.5.0.7.csv')

In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 154097 entries, 0 to 154096
Data columns (total 16 columns):
 #   Column   Non-Null Count   Dtype 
---  ------   --------------   ----- 
 0   title    154097 non-null  object
 1   text     154097 non-null  object
 2   id       154097 non-null  object
 3   date     154097 non-null  object
 4   like     154097 non-null  int64 
 5   rate     154097 non-null  int64 
 6   place    154097 non-null  object
 7   season   154097 non-null  object
 8   cast     154097 non-null  object
 9   show     154097 non-null  object
 10  genre    154097 non-null  object
 11  body     154097 non-null  object
 12  leng     154097 non-null  int64 
 13  rig      154097 non-null  int64 
 14  live     154097 non-null  int64 
 15  subLive  154097 non-null  object
dtypes: int64(5), object(11)
memory usage: 18.8+ MB


In [4]:
T = df[df['rig'] == 1].subLive.unique()  #조작 후기
F = df[df['rig'] == 0].subLive.unique()  #비조작 후기
T_sub_F = [x for x in T if x not in F]  #조작 후기에서 비조작 후기와 같은 내용을 제외

In [5]:
print(len(T), len(F), len(T_sub_F))

6039 115198 5382


In [15]:
from sklearn.feature_extraction.text import TfidfVectorizer 

text = T_sub_F
tfidf_vectorizer = TfidfVectorizer() # TF-IDF 객체선언
tfidf_vectorizer.fit(text) # 단어를 학습시킴 
#tfidf_vectorizer.vocabulary_ # 단어사전을 출력 
sorted(tfidf_vectorizer.vocabulary_.items()) # 단어사전 정렬
tfidf_matrix = tfidf_vectorizer.transform(text).toarray() #결과 행렬 출력
tfidf_matrix

#출처: https://chan-lab.tistory.com/24 [은공지능 공작소]

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

In [17]:
tfidf_matrix.shape

(5382, 31011)

In [16]:
from sklearn.metrics.pairwise import cosine_similarity

cosine_matrix = cosine_similarity(tfidf_matrix, tfidf_matrix)
cosine_matrix.shape

(5382, 5382)

### 조작 후기에서 비조작 후기를 제외한 결과 중 상위 3개에 대하여 높은 코사인 유사도를 갖는 상위 10개의 조작 후기 추출

#### 자주 등장한 조작 후기

In [26]:
most = df[df['rig']==1]['subLive'].value_counts().index.to_list()
most_sub_F = [x for x in most if x not in F]

In [27]:
most_sub_F[0:10]

['시카고 관람 후기 유튜브로만 보던 시카고 공연을 실제로 보아서 눈물이 날 뻔했어요 진짜 13만 원이 아쉽지 않은 공연이었어요 모든 배우분들과 연주하시는 분들 모든 분들이 공연에 최선을 다한다는 것이 보여서 저 역시 박수를 아끼지 않고 봤던 것 같습니다 만약 이 공연을 볼까 말까 고민하신다면 꼭 보는 걸 추천합니다 그리고 미리 시카고 영화를 보고 이 공연을 보시는 걸 추천드립니다 다음에 또 이 공연을 볼 수 있는 기회가 있다면 이번에는 다른 배우님들의 시카고 공연을 보고 싶습니다 앞에서 자고 계신 분들과 재미가 없다는 분들이 계시더군요 저는 정말 이해가 되지 않았습니다 저는 팬카페라도 가입을 해야 하나 하며 정말 5일이 지났지만 여운이 가시지 않는데 어떻게 재미없고 자는 분들이 계신 건지 정말 진심으로 이해가 네버 가지 않네요',
 '정말 후회 없는 관극 김준수 배우님 정말 캐릭터 몰입도가 장난 아니었습니다 노래 연기도 수준급이지만 다른 배우들의 다른 작품도 여럿 봤지만 캐릭터 표현 하나만 따지면 정말 비교할 수 있는 배우가 없다고 생각이 드네요 다음에 다른 드라큘라 배역 캐스팅으로 비교 한번 해보고 싶네요',
 '직접 가서 꼭 보고 싶어요 두 배우 조합의 케미가 미쳤어요 집에서도 재밌게 관람할 수 있었지만 직접 가서 봤으면 더 좋았을 텐데 다른 배우님들 케미도 너무 궁금해요',
 '뮤지컬 드라큘라 뮤지컬 드라큘라 오랜만에 봤는데 이번 뮤지컬 드라큘라 역시 최고의 무대였던 것 같아요 준수오빠의샤큘모습도오랜만에봐서 너무 좋았어요 모든 배우분들 역시 최고의 무대였던 것 같아요',
 '믿고 보는 믿고 보는 최재림 배우 역시 최고 첨부터 끝까지 지루할 틈이 없음',
 '처음은 음 재밌네 오열하긴 함 이상하게 자기 전 계속 생각남 조쉬가 찡해짐 유튜브 찾아봄 다른 페어의 영상을 본 후 흔들리기 시작 한 번 더 볼까 어느새 또 예매하는 나를 발견',
 '말해 뭐 해 말해 뭐 해 너무 완벽한 공연이었어요 드라큘라의 애절함과 감정들을 잘 느낄 수 있었어요 이미 두 번 본 

In [39]:
#딕셔너리 생성
textIdx = {}
for i, c in enumerate(most): 
    textIdx[i] = c
    
idxText = {}
for i, c in textIdx.items(): 
    idxText[c] = i

#### 첫 번째 후기에 대하여 높은 코사인 유사도를 갖는 상위 10개의 조작 후기

In [49]:
#pd.set_option('display.max_colwidth', None)

idx = idxText['시카고 관람 후기 유튜브로만 보던 시카고 공연을 실제로 보아서 눈물이 날 뻔했어요 진짜 13만 원이 아쉽지 않은 공연이었어요 모든 배우분들과 연주하시는 분들 모든 분들이 공연에 최선을 다한다는 것이 보여서 저 역시 박수를 아끼지 않고 봤던 것 같습니다 만약 이 공연을 볼까 말까 고민하신다면 꼭 보는 걸 추천합니다 그리고 미리 시카고 영화를 보고 이 공연을 보시는 걸 추천드립니다 다음에 또 이 공연을 볼 수 있는 기회가 있다면 이번에는 다른 배우님들의 시카고 공연을 보고 싶습니다 앞에서 자고 계신 분들과 재미가 없다는 분들이 계시더군요 저는 정말 이해가 되지 않았습니다 저는 팬카페라도 가입을 해야 하나 하며 정말 5일이 지났지만 여운이 가시지 않는데 어떻게 재미없고 자는 분들이 계신 건지 정말 진심으로 이해가 네버 가지 않네요']
sim_scores = [(i, c) for i, c in enumerate(cosine_matrix[idx]) if i != idx] # 자기 자신을 제외한 영화들의 유사도 및 인덱스를 추출 
sim_scores = sorted(sim_scores, key = lambda x: x[1], reverse=True) # 유사도가 높은 순서대로 정렬
sim_scores = [(textIdx[i], score) for i, score in sim_scores[0:10]]
pd.DataFrame(sim_scores)

,0,1
0,추천합니다 추천합니다,0.245970
1,뮤지컬 드라큘라 뮤지컬 드라큘라 오랜만에 봤는데 이번 뮤지컬 드라큘라 역시 최고의 무대였던 것 같아요 준수오빠의샤큘모습도오랜만에봐서 너무 좋았어요 모든 배우분들 역시 최고의 무대였던 것 같아요,0.227764
2,재밌어요 강추합니다,0.223281
3,재밌어요 재밌어요,0.221307
4,감동입니다 어려운 시국에 공연 올려주셔서 감사해요 덕분에 잠시나마 공연 보면서 위로를 얻고 갑니다,0.218835
5,정말 후회 없는 관극 김준수 배우님 정말 캐릭터 몰입도가 장난 아니었습니다 노래 연기도 수준급이지만 다른 배우들의 다른 작품도 여럿 봤지만 캐릭터 표현 하나만 따지면 정말 비교할 수 있는 배우가 없다고 생각이 드네요 다음에 다른 드라큘라 배역 캐스팅으로 비교 한번 해보고 싶네요,0.192340
6,정말 재밌었어요 맨 앞이라 더 존잼 정말 재밌었어요 맨 앞이라 더 존잼,0.189459
7,위키드 가지 마 못 보내겠어요 너무 사랑하는 이야기가 되어버려서 나 하나 글린다와 옥주현엘파바 둘 다 더블 배우들보다 좀 더 성숙한 캐릭터처럼 느껴져서 밸런스가 잘 맞네요,0.179732
8,좋았어요 넘버도 좋고 다 좋았어요,0.167809
9,추천 추천합니다,0.149316


#### 두 번째 후기에 대하여 높은 코사인 유사도를 갖는 상위 10개의 조작 후기

In [50]:
idx = idxText['정말 후회 없는 관극 김준수 배우님 정말 캐릭터 몰입도가 장난 아니었습니다 노래 연기도 수준급이지만 다른 배우들의 다른 작품도 여럿 봤지만 캐릭터 표현 하나만 따지면 정말 비교할 수 있는 배우가 없다고 생각이 드네요 다음에 다른 드라큘라 배역 캐스팅으로 비교 한번 해보고 싶네요']
sim_scores = [(i, c) for i, c in enumerate(cosine_matrix[idx]) if i != idx] # 자기 자신을 제외한 영화들의 유사도 및 인덱스를 추출 
sim_scores = sorted(sim_scores, key = lambda x: x[1], reverse=True) # 유사도가 높은 순서대로 정렬
sim_scores = [(textIdx[i], score) for i, score in sim_scores[0:10]]
pd.DataFrame(sim_scores)

,0,1
0,추천합니다 추천합니다,0.692725
1,뮤지컬 드라큘라 뮤지컬 드라큘라 오랜만에 봤는데 이번 뮤지컬 드라큘라 역시 최고의 무대였던 것 같아요 준수오빠의샤큘모습도오랜만에봐서 너무 좋았어요 모든 배우분들 역시 최고의 무대였던 것 같아요,0.555994
2,재밌어요 강추합니다,0.545050
3,재밌어요 재밌어요,0.540232
4,추천 추천합니다,0.294845
5,석자 전 붉다 이 궁궐은 참으로 붉다,0.262254
6,재밌다,0.208541
7,강렬한 무대 커튼콜까지 완벽히 통제된 듯한 느낌을 주는 연출 너무 멋있었어요 다시 보고 싶은 공연,0.193428
8,시카고 관람 후기 유튜브로만 보던 시카고 공연을 실제로 보아서 눈물이 날 뻔했어요 진짜 13만 원이 아쉽지 않은 공연이었어요 모든 배우분들과 연주하시는 분들 모든 분들이 공연에 최선을 다한다는 것이 보여서 저 역시 박수를 아끼지 않고 봤던 것 같습니다 만약 이 공연을 볼까 말까 고민하신다면 꼭 보는 걸 추천합니다 그리고 미리 시카고 영화를 보고 이 공연을 보시는 걸 추천드립니다 다음에 또 이 공연을 볼 수 있는 기회가 있다면 이번에는 다른 배우님들의 시카고 공연을 보고 싶습니다 앞에서 자고 계신 분들과 재미가 없다는 분들이 계시더군요 저는 정말 이해가 되지 않았습니다 저는 팬카페라도 가입을 해야 하나 하며 정말 5일이 지났지만 여운이 가시지 않는데 어떻게 재미없고 자는 분들이 계신 건지 정말 진심으로 이해가 네버 가지 않네요,0.192340
9,블루레인 언제나 블루 블루레인 넘버가 기억에 남습니다 배우들 열연 최고,0.177979


#### 세 번째 후기에 대하여 높은 코사인 유사도를 갖는 상위 10개의 조작 후기

In [51]:
idx = idxText['직접 가서 꼭 보고 싶어요 두 배우 조합의 케미가 미쳤어요 집에서도 재밌게 관람할 수 있었지만 직접 가서 봤으면 더 좋았을 텐데 다른 배우님들 케미도 너무 궁금해요']
sim_scores = [(i, c) for i, c in enumerate(cosine_matrix[idx]) if i != idx] # 자기 자신을 제외한 영화들의 유사도 및 인덱스를 추출 
sim_scores = sorted(sim_scores, key = lambda x: x[1], reverse=True) # 유사도가 높은 순서대로 정렬
sim_scores = [(textIdx[i], score) for i, score in sim_scores[0:10]]
pd.DataFrame(sim_scores)

,0,1
0,붉은 정원 추천해요,0.636442
1,좋았어요 역시 조승우,0.619362
2,정말 좋았어요 보고 나오자마자 정말 너무 좋다고 지인한테 막 말했어요 어쩜 모든 배우님이 이렇게 다 잘하시죠 진짜 이번 시즌 본 중에 가장 좋았어요,0.588286
3,진짜짱 최고 가극이랍니다 사랑하고요,0.586878
4,언제 봐도 즐거운 시카고 언제 봐도 즐거운 시카고입니다 음 페어에 따라서 아쉽기도 좋기도 하지만 극 자체가 즐거워요,0.552851
5,항상 최고 최고임,0.542780
6,조형균 배우님 최고입니다 영화를 보고 가서 대사나 내용이 쏙쏙 들어오고 배우들의 연기력 최고입니다 특히 조형균 배우님 정말 반했어요 조형균 파이팅 에델 라인 클랑 파이팅,0.537014
7,난 뭐가 그리울까 그리워할 건 있나 난 무얼 그렸었나 거기 남겨진 내 자화상,0.526013
8,최고 속 시원한 샤우팅에 오늘의 고단함도 날려버리고 갑니다 공연 실황 영상이 나왔으면 좋겠어요 공연 내에 함축된 많은 이야기들을 이 공연이 끝나고도 간직하고 싶어요,0.506744
9,역시 오리지널의 감동이 쓰나미네요 아무래도 자국어가 아니다 보니 몰입이 쉽지는 않으나 왜 프랑스어가 유럽의 사교 어였는지 알 것 같아요 모든 넘버가 빠지지 않고 좋았어요 프롤로 역의 다니엘 아부 아님 오늘부터 팬 되기로 했어요 에스메랄다와 플로르 드리스 가 중앙의 햇살을 보며 부르는 장면 여자배우들이 남자배우들에게 하나도 빠지지 않아요 한국 버전에서는 조금 약하잖아요 역시 길이가 길어서인지 표현력이 칭찬해 3입니다 운이 좋아서 대기 신청되자 직장 빼고 타고 간 보람이 있는 하루였어요,0.497325
